In [8]:
import pandas_datareader  as pdr
import pandas as pd
import datetime
import os

# Replace 'YOUR_API_KEY' with your actual FRED API key
api_key = '94b7d6c1c55f0ba7ea0d3488b2095ed4'

# Set the FRED API key
pdr.fred.FredReader.api_key = api_key

# Specify series IDs
cpi_series_id = 'USACPALTT01CTGYM'
unemployment_series_id = 'UNRATE'
ten_year_treasury_series_id = 'DGS10'

# Fetch CPI data from 2010 to the present
cpi_data = pdr.fred.FredReader(cpi_series_id, start=datetime.datetime(2010, 1, 1)).read()

# Create a DataFrame
cpi_data = pd.DataFrame({
    'Date': cpi_data.index,
    'CPI': cpi_data[cpi_series_id]
})

# Fetch Unemployment Rate data from 2010 to the present
unemployment_data = pdr.fred.FredReader(unemployment_series_id, start=datetime.datetime(2010, 1, 1)).read()

# Create a DataFrame
unemployment_data = pd.DataFrame({
    'Date': unemployment_data.index,
    'Unemployment Rate': unemployment_data[unemployment_series_id]
})

def interpolate_daily_data(data, date_col, value_col):
    """Interpolate daily data from monthly data."""
    # Convert 'Date' to datetime
    data[date_col] = pd.to_datetime(data[date_col])

    # Interpolate daily data
    daily_data = pd.DataFrame()
    # Add the first month's data
    daily_data = pd.concat([daily_data, data.head(1)], ignore_index=True)

    for i in range(len(data) - 1):
        start_date, end_date = data[date_col].iloc[i], data[date_col].iloc[i + 1]
        num_days = (end_date - start_date).days
        roc = (data[value_col].iloc[i + 1] - data[value_col].iloc[i]) / num_days

        # Exclude the end date of the last month
        daily_data = pd.concat([daily_data, pd.DataFrame({
            date_col: [start_date + pd.Timedelta(days=j) for j in range(1, num_days + 1)],
            value_col: data[value_col].iloc[i] + roc * pd.Series(range(1, num_days + 1))
        })], ignore_index=True)

    return daily_data.reset_index(drop=True)

# Interpolate daily CPI data
daily_cpi = interpolate_daily_data(cpi_data, 'Date', "CPI")

# Interpolate daily Unemployment Rate data
daily_unemployment = interpolate_daily_data(unemployment_data, 'Date', "Unemployment Rate")

# Fetch 10-year treasury data from 2010 to the present
ten_yr = pdr.fred.FredReader(ten_year_treasury_series_id, start=datetime.datetime(2010, 1, 1)).read()

# Create a DataFrame
ten_yr_data = pd.DataFrame({
    'Date': ten_yr.index,
    '10-year Treasury': ten_yr[ten_year_treasury_series_id]
})
ten_yr_data.iloc[0, 1] = ten_yr_data.iloc[1, 1]
ten_yr_data["10-year Treasury"].interpolate(method="linear", inplace=True)


ten_yr_data = ten_yr_data.asfreq("D", method="nearest")
ten_yr_data.drop(columns=["Date"], inplace=True)
ten_yr_data = ten_yr_data.reset_index()
ten_yr_data.rename(columns={"DATE": "Date"}, inplace=True)

# Merge daily CPI and Unemployment Rate data

daily_data = pd.merge(daily_cpi, daily_unemployment, on='Date')
daily_data = pd.merge(daily_data, ten_yr_data, on='Date')
daily_data

,Date,CPI,Unemployment Rate,10-year Treasury
0,2010-01-01,-0.160617,9.8,3.85
1,2010-01-02,-0.159320,9.8,3.85
2,2010-01-03,-0.158022,9.8,3.85
3,2010-01-04,-0.156725,9.8,3.85
4,2010-01-05,-0.155428,9.8,3.77
...,...,...,...,...
5017,2023-09-27,3.272447,3.8,4.61
5018,2023-09-28,3.257911,3.8,4.59
5019,2023-09-29,3.243376,3.8,4.59
5020,2023-09-30,3.228841,3.8,4.59


In [9]:
# Save the data
os.makedirs('datasets', exist_ok=True)
daily_data.to_csv('datasets/macroeconomics_data.csv', index=False)